## An SI model to a Tomato Spotted Wilt Virus experiment
In this example we will perform SI model inference using data on the spread of Tomato Spotted Wilt Virus (TSWV) in a greenhouse from Hughes et al. (1997). In this experiment, 520 plants regularly spaced in a 10m x 26m greenhouse were examined for pressence of TSWV once every two weeks. Plants were not removed after showing signs of infection by TSWV. The experiment concluded after 14 weeks, which saw a total of 327 individual plants infected.

### References
* Hughes G, McRoberts N, Madden LV, Nelson SC (1997). “Validating Mathematical Models of Plant-Disease Progress in Space and Time.” Mathematical Medicine and Biology: A Journal of the Institute of Mathematics and Its Applications, 14(2), 85– 112.

## Initialization
Load `Pathogen`, as well as:
* `CSV` for extended CSV file I/O functionality,
* `Distributed`, `Random`, and `DelimitedFiles` from Julia Base,
* `DataFrames` for storing individual level information,
* `Distributions` for specification of priors for Bayesian inference,
* `Plots` for visualization (using whichever visualization backend your prefer), and


In [ ]:
using CSV, Distributed, DelimitedFiles, Distances, LinearAlgebra, Plots, Random
addprocs(3)
@everywhere using DataFrames, Distributions
@everywhere using Pkg
@everywhere Pkg.activate(joinpath(@__DIR__, "../"))
@everywhere using Pathogen

We'll set the seed for random number generation such that our results here are reproducible:

In [ ]:
Random.seed!(5432)

## TSWV Data
We'll import the TSWV, provided as csv files. 

The first CSV file contains X, Y locations in metres for each individual plant in the study. 

The second CSV file contains records of the first day in which each individual was observed as being infected by TSWV. `NaN` indicates that no signs of infection were observed within the 14 week study period.

In [ ]:
# Use CSV.jl for DataFrames I/O
#
# We know the types of the columns, so we'll manually specify those. 
# * Individual IDs are `Int64`
# * X,Y coordinates are `Float64`s
risks = CSV.read(joinpath(@__DIR__, "02_TSWV_locations.csv"), types=[Int64; Float64; Float64])
pop = Population(risks)

# Will precalculate distances
pop.distances = [euclidean([risks[:x][i]; risks[:y][i]], [risks[:x][j]; risks[:y][j]]) for i = 1:pop.individuals, j = 1:pop.individuals]

# Use julia's included CSV interface for simple vector of observation times
raw_observations = readdlm(joinpath(@__DIR__, "02_TSWV_infection_observations.csv"))[:]

# Create an `EventObservations` object with `Pathogen.jl`
obs = EventObservations{SI}(raw_observations)

We will now formulate our `SI` individual level model. For our example, this model will be quite simple as we have a contained artifical environment which limits exogeneous transmissions, and we do not have individual level risk factors to consider beyond basic location data. We will use some common functions which have been prewritten in our examples folder.



In [ ]:
@everywhere include(joinpath(@__DIR__, "risk_functions.jl"))

rf = RiskFunctions{SI}(_zero, # sparks function - we will assume no exogenous transmissions and set this to zero
                       _one, # susceptibility function - we do not have individual level risk factor information to explore here, so will set to a constant 1
                       _powerlaw_w_intercept, # transmissability function - we will use a powerlaw (with intercept) kernel. This provides a spatial and non-spatial component to infection transmissions. This has 3 parameters.
                       _one) # infectivity function - we do not have individual level risk factor information to explore here, so will set to a constant 1

In [ ]:
rpriors = RiskPriors{SI}(UnivariateDistribution[], # empty `UnivariateDistribution` vector for all parameter-less functions
                         UnivariateDistribution[], 
                         [Gamma(10.0, 10.0); Gamma(10.0, 10.0); Gamma(1.0, 1.0)], # Relatively uninformative priors with appropriate support
                         UnivariateDistribution[])

We provide some bounds to event times in comparision to observation times. Actual onset of infectiousness in this study could have occurred any time between plant examinations. This means the observation delay could be up to 14.0 days.

In [ ]:
ee = EventExtents{SI}(14.0)

In [ ]:
mcmc = MCMC(obs, ee, pop, rf, rpriors)
start!(mcmc, 3, attempts = 1000)

In [ ]:
iterate!(mcmc, 5000, 0.25, event_batches = 20)

In [ ]:
plot(mcmc.markov_chains[1].risk_parameters, y_scale=:log10)

In [ ]:
# Visualization of transmission network convergence during MCMC
epidemic_animation = @animate for i = 1:10:991
    plot(mcmc.markov_chains[2].transmission_network[i], 
         mcmc.population, 
         mcmc.markov_chains[2].events[i], 
         100.0, 
         aspect_ratio = :equal)
end
mp4(epidemic_animation, joinpath(@__DIR__, "02_TSWV_epidemic_plot_convergence.mp4"), fps=2)

In [ ]:
# Visualization of a single inferred epidemic sample from MCMC
epidemic_animation = @animate for t in range(0.0, stop=100.0, length=100)
    plot(mcmc.markov_chains[2].transmission_network[end], 
         mcmc.population, 
         mcmc.markov_chains[2].events[end], 
         t, 
         aspect_ratio = :equal)
end
mp4(epidemic_animation, joinpath(@__DIR__, "02_TSWV_epidemic_plot_time.mp4"), fps=2)

In [ ]:
# Visualization of epidemic curve convergence
epidemic_animation = @animate for i = 1:20:981
    plot(mcmc.markov_chains[1].events[i], 0.0, 100.0)
end
mp4(epidemic_animation, joinpath(@__DIR__, "02_TSWV_epidemic_curve_time.mp4"), fps=5)